# API creation

In [27]:
import os
hello_world_script_fie=os.path.join(os.path.pardir,'src','models','hello_world_api.py')

In [28]:
%%writefile $hello_world_script_fie

from flask import Flask,request

app=Flask(__name__)
@app.route('/api',methods=['POST'])

def say_hello():
    data=request.get_json(force=True)
    name=data['name']
    return "Hello {0}".format(name)

if __name__=='__main__':
    app.run(port=10001,debug=True)

Overwriting ..\src\models\hello_world_api.py


In [29]:
#run api in gitbash using python hello_world_api.py in folder in which it resides

In [104]:
import json
import requests

In [105]:
url='http://127.0.0.1:10001/api'
data=json.dumps({'name':'sunny'})
r=requests.post(url,data)

In [106]:
print(r.text)

Hello sunny


## Machine learning api using flask

### Building API

In [176]:
machine_learning_api_script_file=os.path.join(os.path.pardir,'src','models','titanic_api1.py')

In [228]:
%%writefile $machine_learning_api_script_file

from flask import Flask,request
import pickle
import numpy as np
import pandas as pd
import os
import json

app=Flask(__name__)


#load modal and scaler files
model_path=os.path.join(os.path.pardir,os.path.pardir,'models')
model_file_path=os.path.join(model_path,'model1.pkl')
scaler_file_path=os.path.join(model_path,'scaler1.pkl')

# model = pickle.load(f)

#with open(scaler_file_path, 'rb') as f:
#    scaler = pickle.load(f)
model=pickle.load(open(model_file_path,'rb'))
   

columns=[u'Age', u'Fare', u'FamilySize', u'IsMother', u'Ismale', u'Deck_A',\
       u'Deck_B', u'Deck_C', u'Deck_D', u'Deck_E', u'Deck_F', u'Deck_G', u'Deck_Z',\
       u'Pclass_1', u'Pclass_2', u'Pclass_3', u'Title_Lady', u'Title_Master',\
       u'Title_Miss', u'Title_Mr', u'Title_Mrs', u'Title_Officer', u'Title_Sir',\
       u'Fare_Bin_very_low', u'Fare_Bin_low', u'Fare_Bin_high',\
       u'Fare_Bin_very_high', u'Embarked_C', u'Embarked_Q', u'Embarked_S',\
       u'AgeState_Adult', u'AgeState_Child']

@app.route('/api',methods=['POST'])

def make_prediction():
    #read json and convert into json string
    data1=json.dumps(request.get_json(True))
    #create panda dataframe
    df=pd.read_json(data1)
    #extract passengerIds
    passenger_ids=df['PassengerId'].ravel()
    #actual survived values
    actuals=df['Survived'].ravel()
    #extract required columns and convert to matrix
    X=df[columns].as_matrix().astype('float')
    #transform the input
    #X_scaled=scaler.transform(X)
    #make predictions
    predictions=model.predict(X)
    #create response dataframe
    df_response=pd.DataFrame({'PassengerId':passenger_ids,'Predicted':predictions,'Actuals':actuals})
    #return json
    return df_response.to_json()

if __name__=='__main__':
    #host flask app at port 10001
    app.run(port=10002,debug=True)

Overwriting ..\src\models\titanic_api1.py


## Invoke api using request

In [167]:
import os
import pandas as pd
test_data_set_path=os.path.join(os.path.pardir,'data','processed')
test_data_file=os.path.join(test_data_set_path,'train.csv')
train_df=pd.read_csv(test_data_file)
type(train_df)

pandas.core.frame.DataFrame

In [168]:
import numpy as np
test_data=train_df[train_df['Survived']==1][:5]

In [233]:
import requests
def make_api_request(data):
    #url
    url='http://127.0.0.1:10002/api'
    #post req
    r=requests.post(url,data)
    return r.json()


In [234]:
output=make_api_request(test_data.to_json())
print(output)


{'Actuals': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1}, 'PassengerId': {'0': 2, '1': 3, '2': 4, '3': 9, '4': 10}, 'Predicted': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1}}


In [236]:
import io
output1=make_api_request(train_df.to_json())
df_result=pd.read_json(json.dumps(output1))
df_result
#res=pd.read_csv(output1.text)
#res.head()
#output1
#urlData = requests.get(url).content
#rawData = pd.read_csv(io.StringIO(output1.decode('utf-8')))
#rawData.head()

,Actuals,PassengerId,Predicted
0,0,1,0
1,1,2,1
10,1,108,0
100,0,190,0
101,1,20,1
102,1,191,1
103,0,192,0
104,1,193,1
105,1,194,1
106,1,195,1


In [237]:
import numpy as np
np.mean(df_result.Actuals==df_result.Predicted)

0.8372615039281706